## Data Combination and Data Cleaning

**Speech Dataset**

In [ ]:
# Load speech dataframe
import os
import numpy as np
import pandas as pd

sessions = np.arange(25, 76)
data=[]

for session in sessions:
    directory = "./TXT/Session "+str(session)+" - "+str(1945+session)
    for filename in os.listdir(directory):
        f = open(os.path.join(directory, filename), encoding="utf8")
        if filename[0]==".": #ignore hidden files
            continue
        splt = filename.split("_")
        data.append([session, 1945+session, splt[0], f.read()])

        
df_speech = pd.DataFrame(data, columns=['Session','Year','ISO-alpha3 Code','Speech'])

**Country-Name Dataset**

In [ ]:
# Load UNSD dataframe(basically is the country-name dataset)
n = 16 #define the columns

# Load all the data using lineterminator = '\n' to get all the  
# columns that are misplaced because of the ',' inside them
unsd_df = pd.read_csv('UNSD — Methodology.csv', usecols=range(n), lineterminator='\n')
unsd_df

In [ ]:
# Rename last column - remove the space(\s)
unsd_df.rename(columns={'Developed / Developing Countries\r': 'Developed / Developing Countries'}, inplace=True)

In [ ]:
# Complete/Combine the name of the "Country or Area" that was misplaced into  
# the M49 Code column and adjust all other columns

for i,j in unsd_df["M49 Code"].items():
    if(len(j)>3):
        unsd_df.iloc[i, unsd_df.columns.get_loc('Country or Area')] += j
        unsd_df.iloc[i, unsd_df.columns.get_loc('M49 Code')] = unsd_df.iloc[i, unsd_df.columns.get_loc('ISO-alpha2 Code')]
        unsd_df.iloc[i, unsd_df.columns.get_loc('ISO-alpha2 Code')] = unsd_df.iloc[i, unsd_df.columns.get_loc('ISO-alpha3 Code')]
        unsd_df.iloc[i, unsd_df.columns.get_loc('ISO-alpha3 Code')] = unsd_df.iloc[i, unsd_df.columns.get_loc('Least Developed Countries (LDC)')]
        unsd_df.iloc[i, unsd_df.columns.get_loc('Least Developed Countries (LDC)')] = unsd_df.iloc[i, unsd_df.columns.get_loc('Land Locked Developing Countries (LLDC)')]
        unsd_df.iloc[i, unsd_df.columns.get_loc('Land Locked Developing Countries (LLDC)')] = unsd_df.iloc[i, unsd_df.columns.get_loc('Small Island Developing States (SIDS)')]        
        unsd_df.iloc[i, unsd_df.columns.get_loc('Small Island Developing States (SIDS)')] = unsd_df.iloc[i, unsd_df.columns.get_loc('Developed / Developing Countries')]
        unsd_df.iloc[i, unsd_df.columns.get_loc('Developed / Developing Countries')] = "Developing"

In [ ]:
# Merge Speech and Country-Name dataframe
speech_and_unsd_df = pd.merge(unsd_df, df_speech, on="ISO-alpha3 Code")

# Select specific columns to the final Speech and Country-Name dataframe
speech_and_countryName_df = speech_and_unsd_df[['Region Name', 'Country or Area', 'Session', 'Year', 'Speech']].copy()
speech_and_countryName_df

**Happiness Dataset**

In [ ]:
# Load hapiness dataframe
happinessdataframe = pd.read_excel('DataPanelWHR2021C2.xls', index_col=[0,1])
happinessdataframe

In [ ]:
# happinessdataframe rename index from 'Country name' to'Country or Area'
happinessdataframe.index.names = ['Country or Area', 'Year']

In [8]:
# Merge Speech-Country-Name dataframe with the Happiness dataframe 
# on the columns 'Country or Area' and 'Year' 
all_data_df = pd.merge(speech_and_countryName_df, happinessdataframe, left_on=['Country or Area','Year'], right_on=['Country or Area','Year'], right_index=True)

# Create indexes on the columns 'Country or Area' and 'Year' 
all_data_df = all_data_df.set_index(['Country or Area','Year'])

# Create two dataframes one for speeches tokenized, and one for speeches tokennized and FreqDist
all_data_tokenized_df = all_data_df.copy()
all_data_tokenized_FreqDist_df = all_data_df.copy()
data_word_vector_df = all_data_df[["Speech"]].copy()

# This is the unmerged Speech dataset
data_word_vector_df_unmerged = speech_and_countryName_df
data_word_vector_df_unmerged =data_word_vector_df_unmerged.set_index(["Country or Area","Year"])



Download (in case you haven't already done so)

In [ ]:
# import nltk

# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('vader_lexicon')

**OPTION 1) Run this if you want a dataframe merged with happiness**

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.probability import FreqDist
import string

# Create all punctuation string variable
punct = '!"#$%&\'()*+0123456789,’-—./:;<=>?@[\\]^_`{}~[\n]'
# Create a mapping table that will have as key the punctuation and as value an empty string
transtab = str.maketrans(dict.fromkeys(punct, ''))

# Loop through all the cells of "Speech" column
for county_year_index,cell in data_word_vector_df["Speech"].items():
    # Remove all punctuations and convert the text to lowercase
    words = word_tokenize(cell.translate(transtab).lower())
    # Create an array that has all the words that don't give information
    notuseful_words = stopwords.words("english")
    # Create and fill an empty array to gather all the important words of every "Speech" cell
    useful_words = []
    for w in words:
        if (w not in notuseful_words) and (len(w) > 2):
            useful_words.append(w)
    # Fill the dataframe with the text of "Speech" for each cell
    data_word_vector_df["Speech"][county_year_index] = ' '.join(useful_words)


**OPTION 2) You can run this instead of the above if you want to make a dataframe with word count from the unmerged example, it will take some minutes to finish around 2-4**

In [9]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.probability import FreqDist
import string

# Create all punctuation string variable
punct = '!"#$%&\'()*+0123456789,’-—./:;<=>?@[\\]^_`{}~[\n]'
# Create a mapping table that will have as key the punctuation and as value an empty string
transtab = str.maketrans(dict.fromkeys(punct, ''))

# Loop through all the cells of "Speech" column
for county_year_index,cell in data_word_vector_df_unmerged["Speech"].items():
    # Remove all punctuations and convert the text to lowercase
    words = word_tokenize(cell.translate(transtab).lower())
    # Create an array that has all the words that don't give information
    notuseful_words = stopwords.words("english")
    # Create and fill an empty array to gather all the important words of every "Speech" cell
    useful_words = []
    for w in words:
        if (w not in notuseful_words) and (len(w) > 2):
            useful_words.append(w)
    # Fill the dataframe with the text of "Speech" for each cell
    data_word_vector_df_unmerged["Speech"][county_year_index] = ' '.join(useful_words)


<ipython-input-9-95104d1e9512>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_word_vector_df_unmerged["Speech"][county_year_index] = ' '.join(useful_words)


In [12]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()

word_count_df = count_vect.fit_transform(data_word_vector_df["Speech"])
speechOnlyDf = pd.DataFrame(word_count_df.toarray() ,columns= count_vect.get_feature_names())

word_count_df_unmerged = count_vect.fit_transform(data_word_vector_df_unmerged["Speech"])
speechOnlyDfUnmerged = pd.DataFrame.sparse.from_spmatrix(word_count_df_unmerged,columns=count_vect.get_feature_names())


**This is used to create a happines and speech dataframe. It creates a dataframe with the whole speech merged with happiness**\
**You NEED to run this if you want the following cells to play |OR| You can skip some cells below, there are comments to find it**

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.probability import FreqDist
import string

# Create all punctuation string variable
punct = '!"#$%&\'()*+0123456789,’-—./:;<=>?@[\\]^_`{}~[\n]'
# Create a mapping table that will have as key the punctuation and as value an empty string
transtab = str.maketrans(dict.fromkeys(punct, ''))

# Loop through all the cells of "Speech" column
for i,j in all_data_df["Speech"].items():
    # Remove all punctuations and convert the text to lowercase
    words = word_tokenize(j.translate(transtab).lower())
    # Create an array that has all the words that don't give information
    sw = stopwords.words("english")
    # Create and fill an empty array to gather all the important words of every "Speech" cell
    no_sw = []
    for w in words:
        if (w not in sw) and (len(w) > 2):
            no_sw.append(w)
    # Fill the dataframe with the tokenized "Speech" for each cell
    all_data_tokenized_df["Speech"][i] = no_sw
    # Fill the dataframe with the word-count of the tokenized "Speech" for each cell
    all_data_tokenized_FreqDist_df["Speech"][i] = FreqDist(no_sw)

Just some 1 visualization for better understanding and some useful keywords

In [ ]:
all_data_tokenized_FreqDist_df["Speech"][1].plot(20)

**Data Cleaning**

In [ ]:
# Observe the mean values for each numerical column
all_data_tokenized_FreqDist_df.describe()


In [ ]:
# Count how many NaN values we have per column
all_data_tokenized_FreqDist_df.isnull().sum()


In [ ]:
# Here is depicted that the null values are all float64 type 
all_data_tokenized_FreqDist_df.dtypes

Keep one of the two approaches !!!

In [ ]:
# Approach 2

# Remove all NaN values
all_data_tokenized_FreqDist_df =all_data_tokenized_FreqDist_df.dropna()

all_data_tokenized_df =all_data_tokenized_df.dropna()

In [ ]:
# The only column that we should consider if it worthy to remove duplicates is "Session"

# Food for thought
# It is possible that there are two sessions rows with the same session for two different countries

# all_data_tokenized_FreqDist_mean_df = all_data_tokenized_FreqDist_mean_df.drop_duplicates(subset=['Session'])
# len(all_data_tokenized_df)

In [ ]:
# Print the available values in column "Session"
all_data_tokenized_FreqDist_df['Session'].unique()

Removing Outliers

In [ ]:
from scipy import stats

# A basic way to remove outliers with Z-score
# Reference : https://stackoverflow.com/questions/23199796/detect-and-exclude-outliers-in-pandas-data-frame

# I am not sure if we should remove the outliers ????

# I do not think we should remove any outliers

numeric_df = all_data_tokenized_FreqDist_df[['Life Ladder', 'Log GDP per capita', 'Social support', 'Healthy life expectancy at birth', 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption', 'Positive affect', 'Negative affect']].copy()
all_data_tokenized_FreqDist_outliers_df = all_data_tokenized_FreqDist_df[(np.abs(stats.zscore(numeric_df)) < 3).all(axis=1)]


**----SKIP HERE IF YOU DIDN'T RUN THE CELL MENTIONED ABOVE----**

In [13]:
#Join to create a very nice and handy dataframe of all words with index the country and year
countryYearWordsUnmerged = pd.DataFrame()
countryYearWordsUnmerged = speech_and_countryName_df.join(speechOnlyDfUnmerged)


In [14]:
# Drop the speech column cause it contains all the info we dont need anymore
countryYearWordsUnmerged= countryYearWordsUnmerged.drop(["Speech"], axis = 1)

In [15]:
# Index by country and year

countryYearWordsUnmerged.set_index(["Country or Area", "Year"],inplace=True)
countryYearWordsUnmerged.head()

Region Name  Session  aaa  aac  aachen  aacknowledged  \
Country or Area Year                                                         
Algeria         1970      Africa       25    0    0       0              0   
                1971      Africa       26    0    0       0              0   
                1972      Africa       27    0    0       0              0   
                1973      Africa       28    0    0       0              0   
                1974      Africa       29    0    0       0              0   

                      aacrev  aadd  aadda  aaddi  ...  сөйлемек  тhomson  хxi  \
Country or Area Year                              ...                           
Algeria         1970       0     0      0      0  ...         0        0    0   
                1971       0     0      0      0  ...         0        0    0   
                1972       0     0      0      0  ...         0        0    0   
                1973       0     1      0      0  ...         0        0    0   
                1974       0     0      0      0  ...         0        0    0   

                      шмс  шоп  шьа  ьол  қарекет  қылмақ  ﬂagrant  
Country or Area Year                                                
Algeria         1970    0    0    0    0        0       0        0  
                1971    0    0    0    0        0       0        0  
                1972    0    0    0    0        0       0        0  
                1973    0    0    0    0        0       0        0  
                1974    0    0    0    0        0       0        0  

[5 rows x 330678 columns]